## 1.API key

In [2]:
api_key = "YOUR_API_KEY"

## 2.Retrieving audio from youtube video

In [3]:
#!pip install pytube

In [4]:
from pytube import YouTube

video = YouTube('https://www.youtube.com/watch?v=iPj6QKMd8qA&list=PLtqF5YXg7GLmCvTswG32NqQypOuYkPRUE&index=29')
yt = video.streams.get_audio_only()

In [5]:
yt.download()

'e:\\Study files\\Data Scientist & Analysis\\09 - Projects\\Data Science Projects\\Data professor\\transcriber-app\\notebook\\Ep 1 How to create your first Streamlit web app in 1 minute shorts.mp4'

In [6]:
import os

current_dir = os.getcwd()

for file_name in os.listdir(current_dir):
    if file_name.endswith('.mp4'):
        mp4_file = os.path.join(current_dir,file_name)
        print(mp4_file)

e:\Study files\Data Scientist & Analysis\09 - Projects\Data Science Projects\Data professor\transcriber-app\notebook\Ep 1 How to create your first Streamlit web app in 1 minute shorts.mp4


## 3.upload youtube audio file to AssemblyAI

In [7]:
import sys
import time 
import requests

file_name = mp4_file

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as f:
        while True:
            data = f.read(chunk_size)
            if not data:
                break
            yield data

headers = {'authorization':api_key}
response = requests.post("https://api.assemblyai.com/v2/upload",
                       headers=headers,
                       data=read_file(file_name))

In [8]:
audio_url = response.json()['upload_url']
print(audio_url)

https://cdn.assemblyai.com/upload/e5bbcd75-f62c-46db-af15-0db3bdeb3c7c


## 4.Transcribe uploaded audio file

In [9]:
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {
    "audio_url":audio_url
}

headers = {
    "authorization":api_key,
    "content-type":"application/json"
}

transcript_input_response = requests.post(endpoint, headers=headers, json=json)
print(transcript_input_response.json())

{'id': '6jyu9mmvv3-e421-4789-93d9-10f08c5b6994', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/e5bbcd75-f62c-46db-af15-0db3bdeb3c7c', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_dete

## 5.Extract Transcript ID 

In [10]:
transcript_id = transcript_input_response.json()['id']
print(transcript_id)

6jyu9mmvv3-e421-4789-93d9-10f08c5b6994


## 6.Retrieve transcription results

In [18]:
endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers = {
    "authorization":api_key,
}
transcript_output_response =  requests.get(endpoint, headers=headers)

In [19]:
transcript_output_response.json()['status']

'completed'

## 7.Print Transcript text

In [20]:
transcript_output_response.json()['text']

"In this video, I'm going to show you how you could build your very first streamlit web application in Python. So the first thing that you need to do is install streamlit. And so let's create a file called app PY, three quotation opening and a three quotation closing in order to allow it to be in markdown mode. So the hashtag means the h one heading and we're making it bold. And then the second word here, we'll make it in italic and then we'll save it. And then we'll head back to run it streamlit. And then voila. You have created your very first streamlit web application in Python. If your finding value in this video, don't forget to smash the like button and also subscribe to the Data Professor YouTube channel. And I'll see you in the next video."

## 8.Save taranscript text to file

In [21]:
yt_text = open("yt.txt", 'w')
yt_text.write(transcript_output_response.json()['text'])
yt_text.close()

## 9.Save as srt File

In [23]:
srt_endpoint = endpoint+'/srt'
srt_response = requests.get(srt_endpoint,headers=headers)

with open("yt.srt", "w") as _file:
    _file.write(srt_response.text)

## Thanks